A parallel version of XOR using `neat.parallel`.

Since XOR is a simple experiment, a parallel version probably won't run any
faster than the single-process version, due to the overhead of
inter-process communication.

If your evaluation function is what's taking up most of your processing time
(and you should check by using a profiler while running single-process),
you should see a significant performance improvement by evaluating in parallel.

This example is only intended to show how to do a parallel experiment
in neat-python.  
You can of course roll your own parallelism mechanism
or inherit from `ParallelEvaluator` if you need to do something more complicated.


In [24]:
import math
import os
import time
import pickle

import neat
import pandas as pd
import numpy as np

import visualize

In [25]:
data = pd.read_csv('trainRW.csv')
data.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,Var11,Var12
0,8.0,0.180,0.37,0.9,0.049,36.0,109,0.99007,2.89,0.44,12.7,6
1,8.0,0.180,0.37,0.9,0.049,36.0,109,0.99007,2.89,0.44,12.7,6
2,7.3,0.650,0.00,1.2,0.065,15.0,21,0.99460,3.39,0.47,10.0,7
3,5.4,0.835,0.08,1.2,0.046,13.0,93,0.99240,3.57,0.85,13.0,7
4,8.7,0.820,0.02,1.2,0.070,36.0,48,0.99520,3.20,0.58,9.8,5


In [26]:
labels = data.loc[:,'Var12']
labels.head()

0    6
1    6
2    7
3    7
4    5
Name: Var12, dtype: int64

In [27]:
outputs = [ np.array([1 if x == idx-1 else 0 for x in range(10)]) for idx in labels]
predictors = data.loc[:, 'Var1':'Var11']
len(predictors.values[0])

11

In [28]:
def eval_genome(genome, config):
    """
    This function will be run in parallel by ParallelEvaluator.  It takes two
    arguments (a single genome and the genome class configuration data) and
    should return one float (that genome's fitness).
    Note that this function needs to be in module scope for multiprocessing.Pool
    (which is what ParallelEvaluator uses) to find it.  Because of this, make
    sure you check for __main__ before executing any code (as we do here in the
    last few lines in the file), otherwise you'll have made a fork bomb
    instead of a neuroevolution demo. :)
    """
    
    N_total = 350
    
    # Grab some random subset of test data
    test_data = data.sample(n=N_total)
    test_predictors = test_data.loc[:, 'Var1':'Var11'].values
    test_labels = test_data.loc[:,'Var12']
    test_outputs = [np.array([1 if x == idx-1 else 0 for x in range(10)]) for idx in test_labels]

    net = neat.nn.FeedForwardNetwork.create(genome, config)
    N_perfect = 0
    N_near = 0
    N_bad = 0
    
    confidence_err = 0
    
    for xi, xo in zip(test_predictors, test_outputs):
        outputs = net.activate(xi)
#         prediction = output.index(max(output))
        answer = xo.tolist().index(max(xo))
        for idx, output in enumerate(outputs):
            if idx == answer:
                N_perfect += output
            elif np.abs(idx - answer) == 1:
                N_near += output
            else:
                N_bad += output
            
            #total_output = np.sum(output)
            #output_err = np.abs(total_output - 1)
            #confidence_err += output_err
#         error = abs(prediction - answer)
#         if error == 0:
#             N_perfect += 1
#         elif error == 1:
#             N_near += 1
#         else:
#             N_bad += 1
    
    
#     score = (2*N_perfect + N_near - 2*N_bad)/N_total
    score = (2*N_perfect - N_near - 2*N_bad)/(N_total)# - confidence_err/N_total

    return score


In [29]:
def run(config_file):
    # Load configuration.
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)

    # Run for up to 300 generations.
    pe = neat.ParallelEvaluator(4, eval_genome)
    winner = p.run(pe.evaluate, 300)
    
    with open('winner.pickle', 'wb') as f:
        pickle.dump(winner, f)

    # Display the winning genome.
    print('\nBest genome:\n{!s}'.format(winner))

    # Show output of the most fit genome against training data.
    print('\nOutput:')
    winner_net = neat.nn.FeedForwardNetwork.create(winner, config)
    
    # Use whole dataset for validation
    predictors = data.loc[:, 'Var1':'Var11'].values
    labels = data.loc[:,'Var12']
    outputs = [np.array([1 if x == idx-1 else 0 for x in range(10)]) for idx in labels]
    
    # setup confusion matrix
    confusion = np.zeros([10,10])
    
    for xi, xo in zip(predictors, outputs):
        output = winner_net.activate(xi)
        print("input {!r}\nexpected output {!r}\ngot {!r}".format(xi, xo, output))
        expected = xo.index(max(xo))
        prediction = output.index(max(output))
        confusion[expected][prediction] += 1
    
    print(confusion)

    #node_names = {-1:'A', -2: 'B', 0:'A XOR B'}
    visualize.draw_net(config, winner, True)
    visualize.plot_stats(stats, ylog=False, view=True)
    visualize.plot_species(stats, view=True)

In [30]:
# Determine path to configuration file. This path manipulation is
# here so that the script will run successfully regardless of the
# current working directory.
local_dir = os.getcwd()
config_path = os.path.join(local_dir, 'config-feedforward')

In [ ]:
run(config_path)


 ****** Running generation 0 ****** 

Population's average fitness: -7.01874 stdev: 2.26918
Best fitness: -0.65024 - size: (10, 110) - species 1 - id 67
Average adjusted fitness: 0.470
Mean genetic distance 1.226, standard deviation 0.158
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   150     -0.7    0.470     0
Total extinctions: 0
Generation time: 9.034 sec

 ****** Running generation 1 ****** 

Population's average fitness: -4.76414 stdev: 1.90829
Best fitness: -0.17859 - size: (10, 109) - species 1 - id 297
Average adjusted fitness: 0.550
Mean genetic distance 1.348, standard deviation 0.156
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1   150     -0.2    0.550     0
Total extinctions: 0
Generation time: 8.581 sec (8.808 average)

 ****** Running generation 2 ****** 

Population's average fitness: -3.125

Population's average fitness: -0.42648 stdev: 0.97705
Best fitness: 0.58620 - size: (11, 97) - species 1 - id 2678
Average adjusted fitness: 0.757
Mean genetic distance 1.736, standard deviation 0.181
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   18   150      0.6    0.757     0
Total extinctions: 0
Generation time: 8.737 sec (8.872 average)

 ****** Running generation 19 ****** 

Population's average fitness: -0.49555 stdev: 0.99262
Best fitness: 0.56685 - size: (11, 94) - species 1 - id 2914
Average adjusted fitness: 0.793
Mean genetic distance 1.783, standard deviation 0.203
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   19   150      0.6    0.793     1
Total extinctions: 0
Generation time: 8.334 sec (8.768 average)

 ****** Running generation 20 ****** 

Population's average fitness: -0.32179 stdev: 0.93929
Best

Population's average fitness: -0.49852 stdev: 1.16669
Best fitness: 0.53589 - size: (10, 87) - species 1 - id 5396
Average adjusted fitness: 0.807
Mean genetic distance 1.850, standard deviation 0.242
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   36   150      0.5    0.807     1
Total extinctions: 0
Generation time: 8.439 sec (8.786 average)

 ****** Running generation 37 ****** 

Population's average fitness: -0.65500 stdev: 1.26352
Best fitness: 0.59536 - size: (17, 93) - species 1 - id 5224
Average adjusted fitness: 0.754
Mean genetic distance 1.899, standard deviation 0.220
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   37   150      0.6    0.754     2
Total extinctions: 0
Generation time: 8.221 sec (8.758 average)

 ****** Running generation 38 ****** 

Population's average fitness: -0.55001 stdev: 1.10371
Best

Population's average fitness: -0.36854 stdev: 0.95441
Best fitness: 0.53300 - size: (18, 91) - species 1 - id 8115
Average adjusted fitness: 0.820
Mean genetic distance 1.920, standard deviation 0.267
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   54   150      0.5    0.820    15
Total extinctions: 0
Generation time: 11.290 sec (10.233 average)

 ****** Running generation 55 ****** 

Population's average fitness: -0.57701 stdev: 1.13678
Best fitness: 0.59081 - size: (18, 92) - species 1 - id 8222
Average adjusted fitness: 0.719
Mean genetic distance 1.897, standard deviation 0.158
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   55   150      0.6    0.719    16
Total extinctions: 0
Generation time: 11.382 sec (10.509 average)

 ****** Running generation 56 ****** 

Population's average fitness: -0.72528 stdev: 1.18963


In [12]:
winner = pickle.load(open("winner.pickle", "rb"))
winner

In [23]:
# Load configuration.
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)
# Display the winning genome.
print('\nBest genome:\n{!s}'.format(winner))

# Show output of the most fit genome against training data.
print('\nOutput:')
winner_net = neat.nn.FeedForwardNetwork.create(winner, config)

# Use whole dataset for validation
predictors = data.loc[:, 'Var1':'Var11'].values
labels = data.loc[:,'Var12']
outputs = [np.array([1 if x == idx-1 else 0 for x in range(10)]) for idx in labels]

# setup confusion matrix
confusion = np.zeros([10,10])

for xi, xo in zip(predictors, outputs):
    output = winner_net.activate(xi)
    print("input {!r}\nexpected output {!r}\ngot {!r}".format(xi, xo, output))
    expected = xo.tolist().index(max(xo))
    prediction = output.index(max(output))
    confusion[expected][prediction] += 1

print(confusion)

#node_names = {-1:'A', -2: 'B', 0:'A XOR B'}
visualize.draw_net(config, winner, True)
visualize.plot_stats(stats, ylog=False, view=True)
visualize.plot_species(stats, view=True)


Best genome:
Key: 44123
Fitness: -4.41715765457459
Nodes:
	0 DefaultNodeGene(key=0, bias=-0.8633364965991096, response=1.0, activation=sigmoid, aggregation=sum)
	1 DefaultNodeGene(key=1, bias=0.13120219824769908, response=1.0, activation=sigmoid, aggregation=sum)
	2 DefaultNodeGene(key=2, bias=-0.17217915452982002, response=1.0, activation=sigmoid, aggregation=sum)
	3 DefaultNodeGene(key=3, bias=-2.351680811515051, response=1.0, activation=sigmoid, aggregation=sum)
	4 DefaultNodeGene(key=4, bias=-0.6722319241713164, response=1.0, activation=sigmoid, aggregation=sum)
	5 DefaultNodeGene(key=5, bias=-0.01922465578230903, response=1.0, activation=sigmoid, aggregation=sum)
	6 DefaultNodeGene(key=6, bias=-0.17488369169037393, response=1.0, activation=sigmoid, aggregation=sum)
	7 DefaultNodeGene(key=7, bias=1.567894094251642, response=1.0, activation=sigmoid, aggregation=sum)
	8 DefaultNodeGene(key=8, bias=1.9190951015979014, response=1.0, activation=sigmoid, aggregation=sum)
	9 DefaultNodeG

input array([ 8.6    ,  0.685  ,  0.1    ,  1.6    ,  0.092  ,  3.     ,
       12.     ,  0.99745,  3.31   ,  0.65   ,  9.55   ])
expected output array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])
got [0.0, 0.0, 0.0, 0.9999963198758732, 1.0, 1.0, 1.0, 8.75651076269652e-27, 0.0, 0.0]
input array([ 8.7    ,  0.675  ,  0.1    ,  1.6    ,  0.09   ,  4.     ,
       11.     ,  0.99745,  3.31   ,  0.65   ,  9.55   ])
expected output array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
got [0.0, 0.0, 0.0, 0.9999948874808602, 1.0, 1.0, 1.0, 8.75651076269652e-27, 0.0, 0.0]
input array([ 6.2   ,  0.58  ,  0.    ,  1.6   ,  0.065 ,  8.    , 18.    ,
        0.9966,  3.56  ,  0.84  ,  9.4   ])
expected output array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
got [0.0, 0.0, 0.0, 0.9998838735371642, 1.0, 1.0, 1.0, 8.75651076269652e-27, 0.0, 0.0]
input array([7.80e+00, 5.30e-01, 1.00e-02, 1.60e+00, 7.70e-02, 3.00e+00,
       1.90e+01, 9.95e-01, 3.16e+00, 4.60e-01, 9.80e+00])
expected output array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
got [0.0, 

input array([10.9   ,  0.39  ,  0.47  ,  1.8   ,  0.118 ,  6.    , 14.    ,
        0.9982,  3.3   ,  0.75  ,  9.8   ])
expected output array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])
got [0.0, 0.0, 0.0, 0.9434553010696978, 1.0, 1.0, 1.0, 8.75651076269652e-27, 0.0, 0.0]
input array([ 9.1   ,  0.28  ,  0.48  ,  1.8   ,  0.067 , 26.    , 46.    ,
        0.9967,  3.32  ,  1.04  , 10.6   ])
expected output array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])
got [0.0, 0.0, 0.0, 0.30966159606347526, 1.0, 1.0, 1.0, 8.75651076269652e-27, 0.0, 0.0]
input array([ 6.8   ,  0.77  ,  0.    ,  1.8   ,  0.066 , 34.    , 52.    ,
        0.9976,  3.62  ,  0.68  ,  9.9   ])
expected output array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
got [0.0, 0.0, 0.0, 0.9999997749429599, 1.0, 1.0, 1.0, 8.75651076269652e-27, 0.0, 0.0]
input array([ 8.4   ,  0.37  ,  0.53  ,  1.8   ,  0.413 ,  9.    , 26.    ,
        0.9979,  3.06  ,  1.06  ,  9.1   ])
expected output array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])
got [0.0, 0.0, 0.0, 0.8963232376670596, 1.

got [0.0, 0.0, 0.0, 0.9999963198758732, 1.0, 1.0, 1.0, 8.75651076269652e-27, 0.0, 0.0]
input array([ 9.3    ,  0.43   ,  0.44   ,  1.9    ,  0.085  ,  9.     ,
       22.     ,  0.99708,  3.28   ,  0.55   ,  9.5    ])
expected output array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
got [0.0, 0.0, 0.0, 0.9841639949131219, 1.0, 1.0, 1.0, 8.75651076269652e-27, 0.0, 0.0]
input array([ 9.3    ,  0.43   ,  0.44   ,  1.9    ,  0.085  ,  9.     ,
       22.     ,  0.99708,  3.28   ,  0.55   ,  9.5    ])
expected output array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
got [0.0, 0.0, 0.0, 0.9841639949131219, 1.0, 1.0, 1.0, 8.75651076269652e-27, 0.0, 0.0]
input array([10.1    ,  0.45   ,  0.23   ,  1.9    ,  0.082  , 10.     ,
       18.     ,  0.99774,  3.22   ,  0.65   ,  9.3    ])
expected output array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])
got [0.0, 0.0, 0.0, 0.9917315125831152, 1.0, 1.0, 1.0, 8.75651076269652e-27, 0.0, 0.0]
input array([8.6000e+00, 2.2000e-01, 3.6000e-01, 1.9000e+00, 6.4000e-02,
       5.3000e+01, 7.700

got [0.0, 0.0, 0.0, 0.999775905858166, 1.0, 1.0, 1.0, 8.75651076269652e-27, 0.0, 0.0]
input array([6.3000e+00, 9.8000e-01, 1.0000e-02, 2.0000e+00, 5.7000e-02,
       1.5000e+01, 3.3000e+01, 9.9488e-01, 3.6000e+00, 4.6000e-01,
       1.1200e+01])
expected output array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])
got [0.0, 0.0, 0.0, 0.9999999997740265, 1.0, 1.0, 1.0, 8.75651076269652e-27, 0.0, 0.0]
input array([ 9.3    ,  0.655  ,  0.26   ,  2.     ,  0.096  ,  5.     ,
       35.     ,  0.99738,  3.25   ,  0.42   ,  9.6    ])
expected output array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
got [0.0, 0.0, 0.0, 0.9999901331372851, 1.0, 1.0, 1.0, 8.75651076269652e-27, 0.0, 0.0]
input array([ 9.3    ,  0.655  ,  0.26   ,  2.     ,  0.096  ,  5.     ,
       35.     ,  0.99738,  3.25   ,  0.42   ,  9.6    ])
expected output array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
got [0.0, 0.0, 0.0, 0.9999901331372851, 1.0, 1.0, 1.0, 8.75651076269652e-27, 0.0, 0.0]
input array([ 7.     ,  0.57   ,  0.     ,  2.     ,  0.19   , 12.   

got [0.0, 0.0, 0.0, 0.9231382260411735, 1.0, 1.0, 1.0, 8.75651076269652e-27, 0.0, 0.0]
input array([7.7000e+00, 7.1500e-01, 1.0000e-02, 2.1000e+00, 6.4000e-02,
       3.1000e+01, 4.3000e+01, 9.9371e-01, 3.4100e+00, 5.7000e-01,
       1.1800e+01])
expected output array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])
got [0.0, 0.0, 0.0, 0.9999986273994641, 1.0, 1.0, 1.0, 8.75651076269652e-27, 0.0, 0.0]
input array([ 7.6    ,  0.715  ,  0.     ,  2.1    ,  0.068  , 30.     ,
       35.     ,  0.99533,  3.48   ,  0.65   , 11.4    ])
expected output array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])
got [0.0, 0.0, 0.0, 0.9999986273994641, 1.0, 1.0, 1.0, 8.75651076269652e-27, 0.0, 0.0]
input array([ 8.3    ,  0.28   ,  0.48   ,  2.1    ,  0.093  ,  6.     ,
       12.     ,  0.99408,  3.26   ,  0.62   , 12.4    ])
expected output array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0])
got [0.0, 0.0, 0.0, 0.30966159606347526, 1.0, 1.0, 1.0, 8.75651076269652e-27, 0.0, 0.0]
input array([ 8.3    ,  0.28   ,  0.48   ,  2.1    ,  0.093  ,  6. 

input array([ 8.9    ,  0.5    ,  0.21   ,  2.2    ,  0.088  , 21.     ,
       39.     ,  0.99692,  3.33   ,  0.83   , 11.1    ])
expected output array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])
got [0.0, 0.0, 0.0, 0.9983913289423764, 1.0, 1.0, 1.0, 8.75651076269652e-27, 0.0, 0.0]
input array([11.1    ,  0.31   ,  0.53   ,  2.2    ,  0.06   ,  3.     ,
       10.     ,  0.99572,  3.02   ,  0.83   , 10.9    ])
expected output array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0])
got [0.0, 0.0, 0.0, 0.5460044686879445, 1.0, 1.0, 1.0, 8.75651076269652e-27, 0.0, 0.0]
input array([11.1    ,  0.31   ,  0.53   ,  2.2    ,  0.06   ,  3.     ,
       10.     ,  0.99572,  3.02   ,  0.83   , 10.9    ])
expected output array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0])
got [0.0, 0.0, 0.0, 0.5460044686879445, 1.0, 1.0, 1.0, 8.75651076269652e-27, 0.0, 0.0]
input array([ 9.2    ,  0.31   ,  0.36   ,  2.2    ,  0.079  , 11.     ,
       31.     ,  0.99615,  3.33   ,  0.86   , 12.     ])
expected output array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

input array([ 5.2   ,  0.49  ,  0.26  ,  2.3   ,  0.09  , 23.    , 74.    ,
        0.9953,  3.71  ,  0.62  , 12.2   ])
expected output array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])
got [0.0, 0.0, 0.0, 0.9977665897394165, 1.0, 1.0, 1.0, 8.75651076269652e-27, 0.0, 0.0]
input array([ 6.     ,  0.49   ,  0.     ,  2.3    ,  0.068  , 15.     ,
       33.     ,  0.99292,  3.58   ,  0.59   , 12.5    ])
expected output array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])
got [0.0, 0.0, 0.0, 0.9977665897394165, 1.0, 1.0, 1.0, 8.75651076269652e-27, 0.0, 0.0]
input array([6.4000e+00, 4.2000e-01, 9.0000e-02, 2.3000e+00, 5.4000e-02,
       3.4000e+01, 6.4000e+01, 9.9724e-01, 3.4100e+00, 6.8000e-01,
       1.0400e+01])
expected output array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])
got [0.0, 0.0, 0.0, 0.978134971469425, 1.0, 1.0, 1.0, 8.75651076269652e-27, 0.0, 0.0]
input array([6.8000e+00, 6.4000e-01, 3.0000e-02, 2.3000e+00, 7.5000e-02,
       1.4000e+01, 3.1000e+01, 9.9545e-01, 3.3600e+00, 5.8000e-01,
       1.0400e+01])
expecte

NameError: name 'stats' is not defined

In [ ]:
test = np.random.rand(10,10)
